In [12]:
from src.tools import GetResponse, OpenAPIGuideRetriever, SH_NoticeScraper, YouthNoticeScraper
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.docstore.document import Document
from dataclasses import dataclass, field
from typing import List
from src.tools import SH_NoticeScraper
from smolagents import CodeAgent, HfApiModel, ToolCallingAgent, OpenAIServerModel
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter

from dotenv import load_dotenv
from kiwipiepy import Kiwi
import os
from huggingface_hub import login


# Hugging Face 로그인 코드
# def huggingface_login():
#     login(token=os.getenv("HUGGING_FACE_API_KEY"))

# huggingface_login()

load_dotenv()
kiwi = Kiwi()

def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)] 

def list_pdf_files(directory: str) -> list:
    """
    주어진 디렉토리에서 PDF 파일 목록(전체 경로)을 반환합니다.
    """
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.lower().endswith('.pdf')]

def load_pdf_documents(pdf_files: list) -> list:
    """
    PDF 파일 목록을 받아 각 파일을 Document 객체로 변환합니다.
    PyMuPDFLoader를 사용하여 PDF 파일의 모든 페이지 텍스트를 추출하고,
    이를 하나의 Document 객체로 결합합니다.
    """
    documents = []
    for pdf_path in pdf_files:
        try:
            loader = PDFPlumberLoader(pdf_path)
            docs = loader.load()  # 각 페이지별 Document 객체 리스트
            # 모든 페이지 텍스트를 하나로 결합
            full_text = "\n".join([doc.page_content for doc in docs])
            document = Document(page_content=full_text, metadata={"source": pdf_path})
            documents.append(document)
        except Exception as e:
            print(f"Error loading {pdf_path}: {e}")
    return documents
    
smolagent_model = OpenAIServerModel(model_id='gpt-4.1-mini')
openapi_dir = 'docs/openapi/'
pdf_files = list_pdf_files(openapi_dir)
open_api_docs = load_pdf_documents(pdf_files)

LH_tools = [OpenAPIGuideRetriever(open_api_docs), GetResponse()]
LH_imports=["json", "requests"]
    
LH_agent = CodeAgent(
    model = smolagent_model, 
    tools = LH_tools, 
    additional_authorized_imports = LH_imports, 
    max_steps = 6, 
    use_e2b_executor= False,
    verbosity_level = 1
    )

SH_agent = CodeAgent(
    description=
    """
    sh_notice_scraper를 사용해서 웹사이트의 게시판 글 목록을 스크래핑하여, 조건(예: 제목에 '장기미임대' 포함, 기간이 '2024-01-01~2025-05-05')에 맞게 게시글 정보를 정리해서 반환하는 agent.
    """,
    model=smolagent_model,
    tools=[SH_NoticeScraper()],
    max_steps = 6,
    use_e2b_executor=False,
    verbosity_level = 1
    )

YOUTH_agent = ToolCallingAgent(
    description=
    """
    youth_notice_scraper를 사용해서 웹사이트의 게시판 글 목록을 스크래핑하고 조건(예: 제목에 '공공임대' 포함, 기간이 '2024-01-01~2025-05-05', 공고명에 포함되어야 하는 keyword '공공')에 맞게 게시글 정보를 정리해서 반환하는 Tool Calling agent.
    """,
    model=smolagent_model,
    tools=[YouthNoticeScraper()],
    max_steps = 1,
    planning_interval=1,
    verbosity_level = 1
)

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Optional
from dotenv import load_dotenv
from datetime import datetime
from src.prompt import SEARCH_OPTION_SYSTEM_PROMPT, NOTICE_SUPERVISOR_SYSTEM_PROMPT, MAIN_SUPERVISOR_SYSTEM_PROMPT
load_dotenv()

class NoticeSearchOption(BaseModel):
  지역: Optional[str] = Field(None, description="공고문 검색 지역")
  기간: Optional[str] = Field(None, description="공고문 검색 기간")
  청약유형: Optional[str] = Field(None, description="공고문 청약유형 및 종류 - 주택 임대, 매입 임대, 청년안심주택, 공공임대주택, 민간임대주택 등등")
  공고상태 : Optional[str] = Field(None, description="공고문의 공고상태이며 공고중, 접수중, 접수마감 세 가지 중 한개")
  keyword : Optional[str] = Field(None, description="공고 제목에 반드시 포함되어야 할 keyword")
  sh_url: str = Field(..., description="서울주택도시공사 임대 공고 URL")

  def __init__(self, **data):
      data.setdefault("sh_url", "https://housing.seoul.go.kr/site/main/sh/publicLease/list?sc=titl&startDate=&endDate=&sv=")
      super().__init__(**data)

def get_notice_search_option(task_prompt, llm, system_prompt=SEARCH_OPTION_SYSTEM_PROMPT):
  prompt = ChatPromptTemplate.from_messages(
    [
      ("system", system_prompt),
      ("human","아래의 프롬프트에서 검색 옵션 파라미터를 추출하세요. \n {prompt}"),
      ("human", "Tip: Make sure to answer in the correct format, today is {today}"),
    ]
  )
  today = datetime.today()
  StrOutput = llm.with_structured_output(NoticeSearchOption)
  chain = prompt | StrOutput
  params = chain.invoke({"prompt" : task_prompt, "today" : today})
  return params

llm = ChatOpenAI(model="gpt-4.1-mini")
task_prompt = "현재 접수중 청년안심주택, SH 모집공고 찾아줘. agent_name : SH"
#task_prompt = "2024-12-01부터 현재까지 청년안심주택 모집공고와 SH 모집공고를 찾아줘. agent_name : SH"
params = get_notice_search_option(task_prompt, llm, SEARCH_OPTION_SYSTEM_PROMPT)
print(params)


지역='서울' 기간='2025-05-19~2025-06-19' 청약유형='' 공고상태='접수중' keyword='' sh_url='https://housing.seoul.go.kr/site/main/sh/publicLease/list?sc=titl&startDate=&endDate=&sv='


In [3]:
from typing import Literal
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles
from langgraph.checkpoint.memory import MemorySaver

@dataclass
class NoticeInfo:
    공고명: str
    공고URL: str
    공고게시일: str
    모집마감일: Optional[str] = None
    공고상태: Optional[str] = None
    청약유형: Optional[str] = None
    공고지역: Optional[str] = '서울특별시'

@dataclass
class NoticeState:
    task: str
    next : str = ""
    notice_url : str = ""
    search_params: NoticeSearchOption = field(default_factory=NoticeSearchOption)
    LH_notice: List[NoticeInfo] = field(default_factory=list)
    YOUTH_notice: List[NoticeInfo] = field(default_factory=list)
    SH_notice: List[NoticeInfo] = field(default_factory=list)
    # supervisor 가 결정한, 호출할 agent 리스트
    todo: List[str] = field(default_factory=list)
    
def LH_node(state : NoticeState) -> Command:
    search_params = get_notice_search_option(state.task+", agent_name : LH", llm, SEARCH_OPTION_SYSTEM_PROMPT)
    LH_option = {}
    LH_option['조건'] = {
        '지역' : search_params.지역,
        '기간' : search_params.기간,
        '청약유형': search_params.청약유형,
        '공고상태': search_params.공고상태 if search_params.공고상태 == "접수마감" else ""
    }
    
    task = """
    조건(지역, 기간, 청약 유형)을 고려한 "임대 주택" 공고를 찾기 위해 get_response tool을 호출하고
    공고의 공고명, 공고URL, 공고게시일, 모집마감일, 공고상태, 공고유형, 공고지역, 청약유형을 정리해서 json 형태로 답변해.
    청약 유형은 공고명을 보고 판단해서 입력해.
    필터링 코드를 작성할 때 반드시 리스트 컴프리헨션을 사용.
    Only output should be in JSON format that starts with [{{ and ends in }}]. Do not output ```json ``` in the result.
    만약 결과가 없으면 "일치하는 공고문 없음"으로 답변해.
    """
    notices = LH_agent.run(task=task,additional_args=LH_option, stream=False)
    todo_list = state.todo
    if 'LH' in todo_list:
        todo_list.remove('LH')
    return Command(
        goto="supervisor",  
        update={
            "todo" : todo_list,
            'search_params':search_params,
            "LH_notice": notices, 
            }
        )
def SH_node(state : NoticeState) -> Command:
    search_params = get_notice_search_option(state.task+", agent_name : SH", llm, SEARCH_OPTION_SYSTEM_PROMPT)
    SH_option = {}
    SH_option['조건'] = {
        '지역' : search_params.지역,
        '기간' : search_params.기간,
        '청약유형': search_params.청약유형,
        '공고상태': search_params.공고상태,
        'keyword': search_params.keyword,
        'sh_url': "https://housing.seoul.go.kr/site/main/sh/publicLease/list?sc=titl&startDate=&endDate=&sv="
    }
    
    task = f"""
    <STEP 1>: sh_notice_parser를 호출하기위해 max_pages(int) 조건기간을 고려하여 3개월당 1페이지로 값을 설정. 
    keyword로 요청이 없을 경우, tool 호출 결과에서 '공고명'에 '발표', '합격자', '경쟁률', '결과'가 포함된 항목을 제외. STEP 1 코드를 한 번에 작성. </STEP 1>
    <STEP 2>: !important 조건(기간, 청약유형, 공고상태)들을 고려해서 결과를 다시 필터링. '공고게시일'이 기간안에 반드시 포함되어야 함. 필터링 코드는 리스트 컴프리헨션을 사용하고 STEP 2 코드 한번에 작성하고 코드를 작성할 때 함수를 사용하지 말고 코드를 작성.</STEP 2>
    """
    notices = SH_agent.run(task=task, additional_args=SH_option, stream=False)
    
    todo_list = state.todo
    if 'SH' in todo_list:
        todo_list.remove('SH')
    return Command(
        goto="supervisor",  
        update={
            "todo" : todo_list,
            'search_params':search_params,
            "SH_notice": notices,
            }
        )

def YOUTH_node(state : NoticeState) -> Command:
    import ast
    search_params = get_notice_search_option(state.task+", agent_name : YOUTH", llm, SEARCH_OPTION_SYSTEM_PROMPT)
    YOUTH_option = {}
    YOUTH_option['조건'] = {
        '지역' : search_params.지역,
        '기간' : search_params.기간,
        '청약유형': "청년안심주택",
        '공고상태': search_params.공고상태, 
        'keyword': search_params.keyword,    
        }

    task = f"""
    청년안심주택 모집 공고를 youth_notice_scraper을 찾고 조건(지역, 기간)에 맞는 게시글들을 찾아서 결과를 정리해.
            
    Tip 1: max_pages 수는 조건 기간 4개월에 페이지 5개를 고려해서 tool 호출하고 호출에 필요한 search_params는 '조건'임.
    Tip 2: 최종 결과는 tool Observation을 수정없이 JSON 형태로 출력.
    
    Only output should be in JSON format that starts with [{{ and ends in }}]. Do not output ```json ``` in the result.
    """
    notices = ast.literal_eval(next(YOUTH_agent.run(task=task, additional_args=YOUTH_option, stream=True)).observations)
    todo_list = state.todo
    if 'YOUTH' in todo_list:
        todo_list.remove('YOUTH')
    return Command(
        goto="supervisor",  
        update={
            "todo" : todo_list,
            'search_params':search_params,
            "YOUTH_notice": notices,
            }
        )

members = ["LH", "SH", "YOUTH"]
options = members + [END]

class Router(TypedDict):
    """Worker to route to next. If no workers needed, route to FINISH."""

    agents: Literal[*options]
    
def supervisor_node(state: NoticeState, system_prompt=NOTICE_SUPERVISOR_SYSTEM_PROMPT) -> Command:
    if not state.todo:
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": state.task}
        ]
        response = llm.with_structured_output(Router).invoke(messages)
        print(response)
        state.todo = response['agents'] + ['human']
    goto = state.todo.pop(0)
    return Command(goto=goto, update={'todo' : state.todo, 'next' : goto})

from src.tools import NoticeFileScraper

class HumanRequest(BaseModel):
    """Forward the conversation to an expert. Use when you can't assist directly or the user needs assistance that exceeds your authority.
    To use this function, pass the user's 'request' so that an expert can provide appropriate guidance.
    """

    request: str

builder = StateGraph(NoticeState)

builder.add_edge(START, "supervisor")
builder.add_node("supervisor", supervisor_node)
#builder.add_conditional_edges("supervisor", "YOUTH")
for agent_name, node in [("LH", LH_node), ("SH", SH_node), ("YOUTH", YOUTH_node)]:
    builder.add_node(agent_name, node)
    builder.add_edge(agent_name, "supervisor")
     
builder.add_conditional_edges("supervisor", lambda NoticeState: NoticeState.next)     

builder.set_entry_point("supervisor")

memory = MemorySaver()
#graph = builder.compile(checkpointer=memory, interrupt_before=['human'])
graph = builder.compile(checkpointer=memory)

In [ ]:
task = "현재 공고중인 청년안심주택 모집공고와 SH 모집공고를 찾아줘"
config1 = {"configurable": {"thread_id": "user_1"}}
answer = graph.invoke({"task" : task}, config=config1)
print(answer)

{'agents': ['YOUTH', 'SH']}


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 청년안심주택 모집 공고를 youth_notice_scraper을 찾고 조건(지역, 기간)에 맞는 게시글들을 찾아서 결과를 정리해.   │
│                                                                                                                 │
│     Tip 1: max_pages 수는 조건 기간 4개월에 페이지 5개를 고려해서 tool 호출하고 호출에 필요한 search_params는   │
│ '조건'임.                                                                                                       │
│     Tip 2: 최종 결과는 tool Observation을 수정없이 JSON 형태로 출력.                                            │
│                                                                                                                 │
│     Only output should be in JSON format that starts with [{ and ends in }]. Do not output ```json ``` in the   │
│ result.                                                                                                         │
│                                                                                                                 │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'조건': {'지역': '서울', '기간': '2025-05-19~2025-06-19', '청약유형': '청년안심주택', '공고상태': '접수중',    │
│ 'keyword': ''}}.                                                                                                │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4.1-mini ──────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'youth_notice_scraper' with arguments: {'max_pages': 5, 'search_params': {'지역': '서울', '기간': │
│ '2025-05-19~2025-06-19', '청약유형': '청년안심주택', '공고상태': '접수중', 'keyword': ''}}                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: |]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(final_filtered_results)                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: [{'청약유형': '수요자맞춤형', '공고명': '[수요자맞춤형 매입임대주택] 성동구 마장행복마을 추가 
입주자 모집 공고', '공고게시일': '2025-06-13', '공고상태': '접수중', '공고URL': 
'https://www.i-sh.co.kr/main/lay2/program/S1T294C295/www/brd/m_241/view.do?seq=289529', '공고지역': '서울특별시', 
'공고유형': '임대주택'}, {'청약유형': '전세임대', '공고명': '2025년 신혼·신생아(Ⅰ유형) 전세임대 입주자 모집 
공고(2025.6.11.)', '공고게시일': '2025-06-11', '공고상태': '접수중', '공고URL': 
'https://www.i-sh.co.kr/main/lay2/program/S1T294C295/www/brd/m_241/view.do?seq=289479', '공고지역': '서울특별시', 
'공고유형': '임대주택'}]

[Step 2: Duration 2.45 seconds| Input tokens: 12,102 | Output tokens: 769]

{'task': '현재 공고중인 청년안심주택 모집공고와 SH 모집공고를 찾아줘', 'next': 'human', 'search_params': NoticeSearchOption(지역='서울', 기간='2025-05-19~2025-06-19', 청약유형='', 공고상태='접수중', keyword='', sh_url='https://housing.seoul.go.kr/site/main/sh/publicLease/list?sc=titl&startDate=&endDate=&sv='), 'YOUTH_notice': [], 'SH_notice': [{'청약유형': '수요자맞춤형', '공고명': '[수요자맞춤형 매입임대주택] 성동구 마장행복마을 추가 입주자 모집 공고', '공고게시일': '2025-06-13', '공고상태': '접수중', '공고URL': 'https://www.i-sh.co.kr/main/lay2/program/S1T294C295/www/brd/m_241/view.do?seq=289529', '공고지역': '서울특별시', '공고유형': '임대주택'}, {'청약유형': '전세임대', '공고명': '2025년 신혼·신생아(Ⅰ유형) 전세임대 입주자 모집 공고(2025.6.11.)', '공고게시일': '2025-06-11', '공고상태': '접수중', '공고URL': 'https://www.i-sh.co.kr/main/lay2/program/S1T294C295/www/brd/m_241/view.do?seq=289479', '공고지역': '서울특별시', '공고유형': '임대주택'}], 'todo': []}


In [16]:
youth_t = YouthNoticeScraper()
search_params =  {'지역': '서울', '기간': '2025-05-19~2025-06-19', '청약유형': '청년안심주택', '공고상태': '접수중', 'keyword': ''}
youth_t.forward(max_pages=5, search_params= search_params)

[{'공고명': '[민간임대] 동묘앞역 청계로벤하임 추가모집공고',
  '공고게시일': '2025-06-18',
  '모집마감일': '2025-06-22',
  '청약유형': '민간임대 청년안심주택',
  '공고상태': '접수중',
  '공고URL': 'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6250&menuNo=400008',
  '공고지역': '서울특별시',
  '공고유형': '임대주택'},
 {'공고명': '[민간임대] 발산역 센터스퀘어 발산 추가모집공고',
  '공고게시일': '2025-06-18',
  '모집마감일': '2025-06-20',
  '청약유형': '민간임대 청년안심주택',
  '공고상태': '접수중',
  '공고URL': 'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6247&menuNo=400008',
  '공고지역': '서울특별시',
  '공고유형': '임대주택'},
 {'공고명': '[민간임대] 노량진역 더써밋타워 추가모집공고',
  '공고게시일': '2025-06-18',
  '모집마감일': '2025-06-21',
  '청약유형': '민간임대 청년안심주택',
  '공고상태': '접수중',
  '공고URL': 'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6251&menuNo=400008',
  '공고지역': '서울특별시',
  '공고유형': '임대주택'},
 {'공고명': '[민간임대] 건대입구역 더포디엄830 추가모집공고',
  '공고게시일': '2025-06-18',
  '모집마감일': '2025-06-22',
  '청약유형': '민간임대 청년안심주택',
  '공고상태': '접수중',
  '공고URL': 'https://soco.seoul.go.kr/youth/bbs/BMSR00015/view.do?boardId=6248&menu

In [6]:
import json

# answer를 json 형태로 이쁘게 출력하기 위해 NoticeSearchOption을 직렬화 가능하도록 변환
def serialize_notice_search_option(obj):
    if isinstance(obj, NoticeSearchOption):
        return obj.__dict__
    raise TypeError(f"Type {type(obj)} not serializable")

pretty_answer = json.dumps(answer, ensure_ascii=False, indent=4, default=serialize_notice_search_option)
print(pretty_answer)


{
    "task": "현재 공고중인 청년안심주택 모집공고와 SH 모집공고를 찾아줘",
    "next": "human",
    "search_params": {
        "지역": "서울",
        "기간": "2025-05-19~2025-06-19",
        "청약유형": "",
        "공고상태": "접수중",
        "keyword": "",
        "sh_url": "https://housing.seoul.go.kr/site/main/sh/publicLease/list?sc=titl&startDate=&endDate=&sv="
    },
    "YOUTH_notice": [],
    "SH_notice": [
        {
            "청약유형": "수요자맞춤형",
            "공고명": "[수요자맞춤형 매입임대주택] 성동구 마장행복마을 추가 입주자 모집 공고",
            "공고게시일": "2025-06-13",
            "공고상태": "접수중",
            "공고URL": "https://www.i-sh.co.kr/main/lay2/program/S1T294C295/www/brd/m_241/view.do?seq=289529",
            "공고지역": "서울특별시",
            "공고유형": "임대주택"
        },
        {
            "청약유형": "전세임대",
            "공고명": "2025년 신혼·신생아(Ⅰ유형) 전세임대 입주자 모집 공고(2025.6.11.)",
            "공고게시일": "2025-06-11",
            "공고상태": "접수중",
            "공고URL": "https://www.i-sh.co.kr/main/lay2/program/S1T294C295/www/brd/m_241/view.do?seq=289479",
    

In [7]:
import pandas as pd
def convert_notices_to_df(state: NoticeState):
    """공고문 정보들을 DataFrame으로 변환합니다."""
    all_notices = []
    
    # _notice로 끝나는 속성들 찾기
    notice_agency = [attr for attr in state.keys() if attr.endswith('_notice')]
    print(notice_agency)
    # 각 공고 리스트에 대해 처리
    for agency in notice_agency:
        agency_name = agency.split('_')[0]  # 'LH_notice' -> 'LH'
        agency_name = "청년안심주택" if agency_name == "YOUTH" else agency_name
        print(agency, agency_name)
        notices = state[agency]
        
        for notice in notices:
            all_notices.append({
                "공고명": notice.get('공고명', ""),
                "공고게시일": notice.get('공고게시일', ""),
                "모집마감일": notice.get('모집마감일', ""),
                "공고상태": notice.get('공고상태', ""),
                "청약유형": notice.get('청약유형', ""),
                "공고URL": notice.get('공고URL', ""),
                "공고지역": notice.get('공고지역', ""),
                "기관유형": agency_name,
                "공고유형": notice.get('공고유형', "")
            })
    
    # DataFrame 생성
    df = pd.DataFrame(all_notices)
    return df
# 공고문 필터링을 위한 ReAct 에이전트 설정
def df_filter(df):
    """공고문 필터링을 위한 ReAct 에이전트를 설정합니다."""
    
    # 필터링 함수 정의
    def filter_notices(df: pd.DataFrame, search_params: Dict[str, Any]) -> pd.DataFrame:
        """검색 조건에 맞게 공고문을 필터링합니다."""
        filtered_df = df.copy()
        
        # 공고상태 필터링
        if search_params.get('공고상태') == '공고중':
            # '공고중'은 '접수중'과 동일하게 취급
            filtered_df = filtered_df[filtered_df['공고상태'] == '접수중']
        elif search_params.get('공고상태'):
            filtered_df = filtered_df[filtered_df['공고상태'] == search_params['공고상태']]
        
        # 지역 필터링
        if search_params.get('지역'):
            filtered_df = filtered_df[filtered_df['공고지역'].str.contains(search_params['지역'], na=False)]
        
        # 청약유형 필터링
        if search_params.get('청약유형'):
            filtered_df = filtered_df[filtered_df['청약유형'].str.contains(search_params['청약유형'], na=False)]
        
        # 키워드 필터링
        if search_params.get('keyword'):
            filtered_df = filtered_df[filtered_df['공고명'].str.contains(search_params['keyword'], na=False)]
            
        return filtered_df.reset_index(drop=True)
    
    # 도구 정의
    tools = [
        {
            "name": "filter_notices",
            "description": "검색 조건에 맞게 공고문을 필터링합니다.",
            "function": filter_notices
        }
    ]
    
    # ReAct 에이전트 생성
    react_agent = create_react_agent(
        llm=llm,
        tools=tools,
        system_prompt="""
        당신은 임대주택 청약 공고문을 필터링하는 전문가입니다.
        사용자가 지정한 검색 파라미터에 따라 공고문을 필터링해야 합니다.
        
        특히 중요한 조건:
        1. 검색 파라미터의 '공고상태'가 '공고중'인 경우에는 '접수중' 상태의 공고문만 반환해야 합니다.
        2. 다른 파라미터(지역, 청약유형, 키워드 등)도 함께 적용해야 합니다.
        
        결과는 DataFrame 형태로 반환해주세요.
        """
    )
    
    return react_agent

df = convert_notices_to_df(answer)
df

['YOUTH_notice', 'SH_notice']
YOUTH_notice 청년안심주택
SH_notice SH


,공고명,공고게시일,모집마감일,공고상태,청약유형,공고URL,공고지역,기관유형,공고유형
0,[수요자맞춤형 매입임대주택] 성동구 마장행복마을 추가 입주자 모집 공고,2025-06-13,,접수중,수요자맞춤형,https://www.i-sh.co.kr/main/lay2/program/S1T29...,서울특별시,SH,임대주택
1,2025년 신혼·신생아(Ⅰ유형) 전세임대 입주자 모집 공고(2025.6.11.),2025-06-11,,접수중,전세임대,https://www.i-sh.co.kr/main/lay2/program/S1T29...,서울특별시,SH,임대주택


In [8]:
d = {}
d['공고문'] = answer['YOUTH_notice'][:10]
d['search_params'] = answer['search_params']
d

{'공고문': [],
 'search_params': NoticeSearchOption(지역='서울', 기간='2025-05-19~2025-06-19', 청약유형='', 공고상태='접수중', keyword='', sh_url='https://housing.seoul.go.kr/site/main/sh/publicLease/list?sc=titl&startDate=&endDate=&sv=')}

In [9]:
df_agent = CodeAgent(
    model = smolagent_model, 
    tools = [], 
    additional_authorized_imports = ["pandas"],
    max_steps = 6, 
    use_e2b_executor= False,
    verbosity_level = 1
    )

task = f"""
공고문을 보고 search_params에 맞는 공고문만 남겨줘 결과를 필터링할 땐 리스트 컴프리헨션을 사용하고 모든 조건 고려하도록 코드 작성"""
notices = df_agent.run(task=task, additional_args=d, stream=False)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ 공고문을 보고 search_params에 맞는 공고문만 남겨줘 결과를 필터링할 땐 리스트 컴프리헨션을 사용하고 모든 조건    │
│ 고려하도록 코드 작성                                                                                            │
│ You have been provided with these additional arguments, that you can access using the keys as variables in your │
│ python code:                                                                                                    │
│ {'공고문': [], 'search_params': NoticeSearchOption(지역='서울', 기간='2025-05-19~2025-06-19', 청약유형='',      │
│ 공고상태='접수중', keyword='',                                                                                  │
│ sh_url='https://housing.seoul.go.kr/site/main/sh/publicLease/list?sc=titl&startDate=&endDate=&sv=')}.           │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4.1-mini ──────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  from datetime import datetime                                                                                    
                                                                                                                   
  # Sample variables given from the context                                                                        
  notices = 공고문                                                                                                 
  params = search_params                                                                                           
                                                                                                                   
  # Parse the 기간 (period)                                                                                        
  period_range = params.기간.split("~")                                                                            
  start_date = datetime.strptime(period_range[0], "%Y-%m-%d") if period_range[0] else None                         
  end_date = datetime.strptime(period_range[1], "%Y-%m-%d") if period_range[1] else None                           
                                                                                                                   
  def date_within_range(notice_date_str):                                                                          
      # Assuming notice date is in 'YYYY-MM-DD' format in each notice under key '날짜'                             
      try:                                                                                                         
          notice_date = datetime.strptime(notice_date_str, "%Y-%m-%d")                                             
          if start_date and notice_date < start_date:                                                              
              return False                                                                                         
          if end_date and notice_date > end_date:                                                                  
              return False                                                                                         
          return True                                                                                              
      except:                                                                                                      
          return False                                                                                             
                                                                                                                   
  filtered_notices = [                                                                                             
      notice for notice in notices                                                                                 
      if (params.지역 == "" or notice.get("지역") == params.지역)                                                  
      and (params.청약유형 == "" or notice.get("청약유형", "") == params.청약유형)                                 
      and (params.공고상태 == "" or notice.get("공고상태", "") == params.공고상태)                                 
      and (params.keyword == "" or params.keyword in notice.get("title", "") or params[38;2;2

Execution logs:
[]

Out: None

[Step 0: Duration 10.64 seconds| Input tokens: 2,101 | Output tokens: 633]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(filtered_notices)                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: []

[Step 1: Duration 1.49 seconds| Input tokens: 5,248 | Output tokens: 689]

In [11]:
answer['filtered_notice'] = notices
filtered_df = convert_notices_to_df(answer)
filtered_df

['YOUTH_notice', 'SH_notice', 'filtered_notice']
YOUTH_notice 청년안심주택
SH_notice SH
filtered_notice filtered


,공고명,공고게시일,모집마감일,공고상태,청약유형,공고URL,공고지역,기관유형,공고유형
0,[수요자맞춤형 매입임대주택] 성동구 마장행복마을 추가 입주자 모집 공고,2025-06-13,,접수중,수요자맞춤형,https://www.i-sh.co.kr/main/lay2/program/S1T29...,서울특별시,SH,임대주택
1,2025년 신혼·신생아(Ⅰ유형) 전세임대 입주자 모집 공고(2025.6.11.),2025-06-11,,접수중,전세임대,https://www.i-sh.co.kr/main/lay2/program/S1T29...,서울특별시,SH,임대주택


In [ ]:
from typing import TypedDict, List, Optional, Annotated
from dataclasses import dataclass, field
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import BaseMessage
import operator

# 데이터 클래스 정의
@dataclass
class NoticeInfo:
    공고명: str
    공고URL: str
    공고게시일: str
    모집마감일: Optional[str] = None
    공고상태: Optional[str] = None
    청약유형: Optional[str] = None
    공고지역: Optional[str] = '서울특별시'

@dataclass
class NoticeSearchOption:
    지역: Optional[str] = None
    기간: Optional[str] = None
    청약유형: Optional[str] = None
    공고상태: Optional[str] = None
    keyword: Optional[str] = None
    sh_url: str = "https://housing.seoul.go.kr/site/main/sh/publicLease/list?sc=titl&startDate=&endDate=&sv="

# 상태 클래스 정의
class NoticeState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    task: str  # 현재 사용자 입력/요청
    next: str  # 다음 이동할 노드
    notice_url: str  # 선택된 공고문 URL
    search_params: NoticeSearchOption  # 검색 매개변수
    LH_notice: List[NoticeInfo]  # LH 스크래핑 결과
    YOUTH_notice: List[NoticeInfo]  # 청년안심주택 스크래핑 결과
    SH_notice: List[NoticeInfo]  # SH 스크래핑 결과
    todo: List[str]  # 호출할 에이전트 리스트
    selected_notice: Optional[NoticeInfo]  # 사용자가 선택한 공고문
    notices_displayed: bool  # 공고문 리스트 표시 여부
    waiting_for_selection: bool  # 사용자 선택 대기 상태

# ==================== Main Graph 노드들 ====================

def main_supervisor_node(state: NoticeState):
    """Main Supervisor: 사용자 입력 분석하여 적절한 서브그래프로 라우팅"""
    print("🎯 Main Supervisor: 사용자 입력 분석 중...")
    
    task = state.get("task", "")
    
    # 공고문 검색/스크래핑 요청인지 확인
    if any(keyword in task.lower() for keyword in ["공고", "검색", "찾아", "스크래핑", "lh", "sh", "청년"]):
        return {"next": "notice_scrap"}
    
    # 선택된 공고문에 대한 질문인지 확인
    elif state.get("selected_notice") and any(keyword in task.lower() for keyword in ["자격", "요건", "신청", "임대료", "보증금"]):
        return {"next": "notice_rag"}
    
    # 일반 청약 지식 질문
    else:
        return {"next": "knowledge_rag"}

def route_to_subgraph(state: NoticeState):
    """라우팅 결정에 따라 다음 노드 결정"""
    next_node = state.get("next", "")
    return next_node

# ==================== Subgraph 1: Notice Scrap ====================

def notice_scrap_supervisor_node(state: NoticeState):
    """Notice Scrap Supervisor: 어떤 스크래핑 에이전트를 호출할지 결정"""
    print("📋 Notice Scrap Supervisor: 스크래핑 에이전트 선택 중...")
    
    task = state.get("task", "").lower()
    todo = []
    
    if "lh" in task or "한국토지주택공사" in task:
        todo.append("LH_node")
    elif "sh" in task or "서울주택도시공사" in task:
        todo.append("SH_node")
    elif "청년" in task or "youth" in task:
        todo.append("YOUTH_node")
    else:
        # 기관 명시가 없으면 모든 기관 검색
        todo = ["LH_node", "SH_node", "YOUTH_node"]
    
    return {"todo": todo}

def LH_node(state: NoticeState):
    """LH(한국토지주택공사) 공고문 스크래핑"""
    print("🏢 LH Node: 한국토지주택공사 공고문 스크래핑 중...")
    
    # 실제 스크래핑 로직 (여기서는 더미 데이터)
    dummy_notices = [
        NoticeInfo(
            공고명="LH 청년전세임대주택 2024년 1차 공고",
            공고URL="http://lh.co.kr/notice/20240115",
            공고게시일="2024-01-15",
            모집마감일="2024-02-15",
            공고상태="모집중",
            청약유형="전세임대",
            공고지역="서울특별시"
        ),
        NoticeInfo(
            공고명="LH 행복주택 입주자 모집 공고",
            공고URL="http://lh.co.kr/notice/20240118",
            공고게시일="2024-01-18",
            모집마감일="2024-02-18",
            공고상태="모집중",
            청약유형="행복주택",
            공고지역="서울특별시"
        )
    ]
    
    print(f"✅ LH에서 {len(dummy_notices)}개 공고문 수집 완료")
    return {"LH_notice": dummy_notices}

def SH_node(state: NoticeState):
    """SH(서울주택도시공사) 공고문 스크래핑"""
    print("🏙️ SH Node: 서울주택도시공사 공고문 스크래핑 중...")
    
    # 실제 스크래핑 로직 (여기서는 더미 데이터)
    dummy_notices = [
        NoticeInfo(
            공고명="SH 청년주택 2024년 상반기 공급 공고",
            공고URL="http://sh.co.kr/notice/20240120",
            공고게시일="2024-01-20",
            모집마감일="2024-02-20",
            공고상태="모집중",
            청약유형="청년주택",
            공고지역="서울특별시"
        ),
        NoticeInfo(
            공고명="SH 공공임대주택 입주자 모집",
            공고URL="http://sh.co.kr/notice/20240122",
            공고게시일="2024-01-22",
            모집마감일="2024-02-22",
            공고상태="모집중",
            청약유형="공공임대",
            공고지역="서울특별시"
        )
    ]
    
    print(f"✅ SH에서 {len(dummy_notices)}개 공고문 수집 완료")
    return {"SH_notice": dummy_notices}

def YOUTH_node(state: NoticeState):
    """청년안심주택 공고문 스크래핑"""
    print("👥 YOUTH Node: 청년안심주택 공고문 스크래핑 중...")
    
    # 실제 스크래핑 로직 (여기서는 더미 데이터)
    dummy_notices = [
        NoticeInfo(
            공고명="청년안심주택 2024년 1차 입주자 모집",
            공고URL="http://youth.housing.kr/notice/20240125",
            공고게시일="2024-01-25",
            모집마감일="2024-02-25",
            공고상태="모집중",
            청약유형="안심주택",
            공고지역="서울특별시"
        ),
        NoticeInfo(
            공고명="청년안심주택 강남구 신규 공급",
            공고URL="http://youth.housing.kr/notice/20240127",
            공고게시일="2024-01-27",
            모집마감일="2024-02-27",
            공고상태="모집중",
            청약유형="안심주택",
            공고지역="서울특별시 강남구"
        )
    ]
    
    print(f"✅ YOUTH에서 {len(dummy_notices)}개 공고문 수집 완료")
    return {"YOUTH_notice": dummy_notices}

def display_notices_node(state: NoticeState):
    """스크래핑된 공고문 목록 표시"""
    print("📊 Display Notices: 공고문 목록 표시 중...")
    
    all_notices = []
    all_notices.extend(state.get("LH_notice", []))
    all_notices.extend(state.get("SH_notice", []))
    all_notices.extend(state.get("YOUTH_notice", []))
    
    print(f"총 {len(all_notices)}개의 공고문을 찾았습니다.")
    for i, notice in enumerate(all_notices, 1):
        print(f"{i}. [{notice.청약유형}] {notice.공고명} ({notice.공고지역})")
        print(f"   📅 {notice.공고게시일} ~ {notice.모집마감일}")
    
    return {"notices_displayed": True, "waiting_for_selection": True}

def process_selection_node(state: NoticeState):
    """사용자 공고문 선택 처리"""
    print("✅ Process Selection: 사용자 선택 처리 중...")
    
    # 실제로는 사용자 입력에서 선택한 번호를 파싱
    # 여기서는 첫 번째 공고를 선택했다고 가정
    all_notices = []
    all_notices.extend(state.get("LH_notice", []))
    all_notices.extend(state.get("SH_notice", []))
    all_notices.extend(state.get("YOUTH_notice", []))
    
    if all_notices:
        selected_notice = all_notices[0]
        print(f"선택된 공고: {selected_notice.공고명}")
        return {"selected_notice": selected_notice, "waiting_for_selection": False}
    
    return {"waiting_for_selection": False}

# ==================== Subgraph 2: Knowledge RAG ====================

def knowledge_retrieval_node(state: NoticeState):
    """일반 지식 검색 (청약 제도, 정책 등)"""
    print("📚 Knowledge Retrieval: 일반 지식 검색 중...")
    
    task = state.get("task", "")
    print(f"지식 검색 쿼리: {task}")
    
    # 실제로는 벡터 DB나 지식 베이스에서 검색
    knowledge_info = {
        "청약 자격": "만 19세 이상, 무주택자, 소득 기준 충족",
        "청약 절차": "1. 신청서 작성 → 2. 서류 제출 → 3. 심사 → 4. 당첨자 발표",
        "필요 서류": "주민등록등본, 소득증명서, 재직증명서 등"
    }
    
    return {"knowledge_result": knowledge_info}

def knowledge_processing_node(state: NoticeState):
    """지식 정보 처리 및 분석"""
    print("🔍 Knowledge Processing: 지식 정보 처리 중...")
    
    # 검색된 지식을 사용자 질문에 맞게 가공
    processed_info = "청약 관련 일반 지식을 정리하여 제공합니다."
    print(f"처리된 지식: {processed_info}")
    
    return state

def knowledge_response_generation_node(state: NoticeState):
    """지식 기반 응답 생성"""
    print("💡 Knowledge Response: 지식 기반 응답 생성 중...")
    
    # 일반 지식을 바탕으로 한 응답 생성
    response = "청약 제도에 대한 일반적인 정보를 제공해드립니다."
    print(f"지식 기반 응답: {response}")
    
    return state

# ==================== Subgraph 3: Notice RAG ====================

def notice_retrieval_node(state: NoticeState):
    """선택된 공고문 내용 검색"""
    print("📋 Notice Retrieval: 공고문 내용 검색 중...")
    
    selected_notice = state.get("selected_notice")
    if selected_notice:
        print(f"공고문 분석 중: {selected_notice.공고명}")
        print(f"URL: {selected_notice.공고URL}")
        # 실제로는 URL에서 전체 공고문 내용을 가져와서 벡터화
    
    return state

def notice_query_processing_node(state: NoticeState):
    """공고문 관련 사용자 질의 처리"""
    print("❓ Notice Query Processing: 공고문 질의 분석 중...")
    
    task = state.get("task", "")
    selected_notice = state.get("selected_notice")
    
    if selected_notice:
        print(f"질문: {task}")
        print(f"대상 공고: {selected_notice.공고명}")
        
        # 질문 유형 분류
        if any(keyword in task.lower() for keyword in ["자격", "조건"]):
            query_type = "자격 요건"
        elif any(keyword in task.lower() for keyword in ["신청", "방법"]):
            query_type = "신청 방법"
        elif any(keyword in task.lower() for keyword in ["임대료", "보증금", "비용"]):
            query_type = "비용 정보"
        else:
            query_type = "일반 정보"
        
        print(f"질문 유형: {query_type}")
    
    return state

def notice_response_generation_node(state: NoticeState):
    """공고문 기반 맞춤형 응답 생성"""
    print("📝 Notice Response: 공고문 기반 응답 생성 중...")
    
    selected_notice = state.get("selected_notice")
    if selected_notice:
        # 실제로는 LLM을 사용하여 공고문 내용과 질문을 종합한 응답 생성
        response = f"{selected_notice.공고명}에 대한 상세 정보를 제공해드리겠습니다."
        print(f"공고문 기반 응답: {response}")
    
    return state

# ==================== 그래프 생성 및 실행 ====================

def create_main_graph():
    """메인 그래프 생성"""
    
    # Subgraph 1: Notice Scrap (공고문 스크래핑)
    notice_scrap_graph = StateGraph(NoticeState)
    notice_scrap_graph.add_node("supervisor", notice_scrap_supervisor_node)
    notice_scrap_graph.add_node("LH", LH_node)
    notice_scrap_graph.add_node("SH", SH_node)
    notice_scrap_graph.add_node("YOUTH", YOUTH_node)
    notice_scrap_graph.add_node("display_notices", display_notices_node)
    notice_scrap_graph.add_node("process_selection", process_selection_node)
    
    # Notice Scrap 연결
    notice_scrap_graph.add_edge(START, "supervisor")
    notice_scrap_graph.add_conditional_edges(
        "supervisor",
        lambda state: state.get("todo", []),
        {
            "LH_node": "LH",
            "SH_node": "SH", 
            "YOUTH_node": "YOUTH"
        }
    )
    notice_scrap_graph.add_edge("LH", "display_notices")
    notice_scrap_graph.add_edge("SH", "display_notices")
    notice_scrap_graph.add_edge("YOUTH", "display_notices")
    notice_scrap_graph.add_edge("display_notices", "process_selection")
    notice_scrap_graph.add_edge("process_selection", END)
    
    # Subgraph 2: Knowledge RAG (일반 지식)
    knowledge_rag_graph = StateGraph(NoticeState)
    knowledge_rag_graph.add_node("retrieval", knowledge_retrieval_node)
    knowledge_rag_graph.add_node("processing", knowledge_processing_node)
    knowledge_rag_graph.add_node("generation", knowledge_response_generation_node)
    
    # Knowledge RAG 연결
    knowledge_rag_graph.add_edge(START, "retrieval")
    knowledge_rag_graph.add_edge("retrieval", "processing")
    knowledge_rag_graph.add_edge("processing", "generation")
    knowledge_rag_graph.add_edge("generation", END)
    
    # Subgraph 3: Notice RAG (공고문 기반)
    notice_rag_graph = StateGraph(NoticeState)
    notice_rag_graph.add_node("retrieval", notice_retrieval_node)
    notice_rag_graph.add_node("query_processing", notice_query_processing_node)
    notice_rag_graph.add_node("generation", notice_response_generation_node)
    
    # Notice RAG 연결
    notice_rag_graph.add_edge(START, "retrieval")
    notice_rag_graph.add_edge("retrieval", "query_processing")
    notice_rag_graph.add_edge("query_processing", "generation")
    notice_rag_graph.add_edge("generation", END)
    
    # 메인 그래프 생성
    main_graph = StateGraph(NoticeState)
    main_graph.add_node("main_supervisor", main_supervisor_node)
    main_graph.add_node("notice_scrap", notice_scrap_graph.compile())
    main_graph.add_node("knowledge_rag", knowledge_rag_graph.compile())
    main_graph.add_node("notice_rag", notice_rag_graph.compile())
    
    # 메인 그래프 연결
    main_graph.add_edge(START, "main_supervisor")
    main_graph.add_conditional_edges(
        "main_supervisor",
        route_to_subgraph,
        {
            "notice_scrap": "notice_scrap",
            "knowledge_rag": "knowledge_rag",
            "notice_rag": "notice_rag"
        }
    )
    main_graph.add_edge("notice_scrap", END)
    main_graph.add_edge("knowledge_rag", END)
    main_graph.add_edge("notice_rag", END)
    
    return main_graph.compile(checkpointer=MemorySaver())

    subgraph3.add_edge(START, "chat")
    subgraph3.add_edge("chat", "info_provider")
    subgraph3.add_edge("info_provider", END)
    
    # 메인 그래프 생성
    main_graph = StateGraph(NoticeState)
    main_graph.add_node("main_supervisor", main_supervisor_node)
    main_graph.add_node("subgraph1", subgraph1.compile())
    main_graph.add_node("subgraph2", subgraph2.compile())
    main_graph.add_node("subgraph3", subgraph3.compile())
    
    # 메인 그래프 연결
    main_graph.add_edge(START, "main_supervisor")
    main_graph.add_conditional_edges(
        "main_supervisor",
        route_to_subgraph,
        {
            "subgraph1": "subgraph1",
            "subgraph2": "subgraph2",
            "subgraph3": "subgraph3"
        }
    )
    main_graph.add_edge("subgraph1", END)
    main_graph.add_edge("subgraph2", END)
    main_graph.add_edge("subgraph3", END)
    
    return main_graph.compile(checkpointer=MemorySaver())

# ==================== 실행 예시 ====================

def run_example():
    """실행 예시"""
    
    # 그래프 생성
    app = create_main_graph()
    
    print("=" * 60)
    print("🏠 임대 주택 청약 도우미 시스템 시작")
    print("=" * 60)
    
    # 테스트 시나리오 1: 공고문 검색
    print("\n📋 시나리오 1: 공고문 검색")
    print("-" * 40)
    
    initial_state = {
        "messages": [],
        "task": "최근 서울 지역 청년주택 공고 찾아줘",
        "next": "",
        "notice_url": "",
        "search_params": NoticeSearchOption(),
        "LH_notice": [],
        "YOUTH_notice": [],
        "SH_notice": [],
        "todo": [],
        "selected_notice": None,
        "notices_displayed": False,
        "waiting_for_selection": False
    }
    
    config = {"configurable": {"thread_id": "example1"}}
    
    for event in app.stream(initial_state, config=config):
        for node_name, node_output in event.items():
            print(f"✅ {node_name} 완료")
    
    print("\n" + "=" * 60)
    print("시스템 실행 완료!")
    print("=" * 60)

# 그래프 시각화 함수들
def visualize_graph():
    """다양한 방법으로 그래프 시각화"""
    app = create_main_graph()
    
    print("\n🎨 LangGraph 시각화 방법들:")
    print("=" * 60)
    
    # 방법 1: draw_mermaid() - 웹 브라우저 없이
    try:
        print("\n📊 방법 1: Mermaid 다이어그램 (텍스트)")
        print("-" * 40)
        mermaid_code = app.get_graph().draw_mermaid()
        print(mermaid_code)
    except Exception as e:
        print(f"❌ Mermaid 오류: {e}")
    
    # 방법 2: draw_ascii() - ASCII 다이어그램
    try:
        print("\n📋 방법 2: ASCII 다이어그램")
        print("-" * 40)
        ascii_diagram = app.get_graph().draw_ascii()
        print(ascii_diagram)
    except Exception as e:
        print(f"❌ ASCII 오류: {e}")
    
    # 방법 3: 그래프 정보 직접 출력
    try:
        print("\n🔍 방법 3: 그래프 구조 분석")
        print("-" * 40)
        graph = app.get_graph()
        
        print("📊 노드 목록:")
        for node in graph.nodes:
            print(f"  🔸 {node}")
        
        print("\n🔗 엣지 연결:")
        for edge in graph.edges:
            print(f"  {edge.source} → {edge.target}")
        
        print("\n🎯 조건부 엣지:")
        for node_id, conditions in graph.branches.items():
            print(f"  📍 {node_id}:")
            for condition, target in conditions.items():
                if isinstance(target, str):
                    print(f"    ↳ {condition} → {target}")
                elif isinstance(target, list):
                    for t in target:
                        print(f"    ↳ {condition} → {t}")
        
    except Exception as e:
        print(f"❌ 구조 분석 오류: {e}")

def save_graph_visualization():
    """그래프 시각화를 파일로 저장"""
    app = create_main_graph()
    
    print("\n💾 그래프 시각화 파일 저장:")
    print("-" * 40)
    
    try:
        # Mermaid 파일로 저장
        mermaid_code = app.get_graph().draw_mermaid()
        with open("graph_visualization.mmd", "w", encoding="utf-8") as f:
            f.write(mermaid_code)
        print("✅ Mermaid 파일 저장: graph_visualization.mmd")
        
        # 사용법 안내
        print("\n📝 Mermaid 시각화 사용법:")
        print("1. https://mermaid.live/ 접속")
        print("2. graph_visualization.mmd 파일 내용 복사")
        print("3. 온라인 에디터에 붙여넣기")
        print("4. 시각적 다이어그램 확인")
        
    except Exception as e:
        print(f"❌ 파일 저장 오류: {e}")

def create_custom_visualization():
    """커스텀 시각화 생성"""
    print("\n🎨 커스텀 그래프 시각화:")
    print("=" * 60)
    
    # 시각적 그래프 표현
    graph_structure = """
    ┌─────────────────────────────────────────────────────────────────────┐
    │                           🏠 임대 주택 청약 도우미                    │
    │                              LangGraph 시스템                       │
    └──────────────────────────┬──────────────────────────────────────────┘
                               │
                               ▼
    ┌─────────────────────────────────────────────────────────────────────┐
    │                    🎯 Main Supervisor                               │
    │                   (사용자 입력 분석 & 라우팅)                         │
    └─────────┬─────────────────┬─────────────────┬─────────────────────────┘
              │                 │                 │
              ▼                 ▼                 ▼
    ┌─────────────────┐ ┌─────────────────┐ ┌─────────────────┐
    │   📋 Subgraph 1  │ │   🔍 Subgraph 2  │ │   💭 Subgraph 3  │
    │   공고문 스크래핑  │ │   RAG 파이프라인  │ │   대화 처리      │
    │                 │ │                 │ │                 │
    │ ┌─────────────┐ │ │ ┌─────────────┐ │ │ ┌─────────────┐ │
    │ │ Supervisor  │ │ │ │  Retrieve   │ │ │ │    Chat     │ │
    │ └─────────────┘ │ │ └─────────────┘ │ │ └─────────────┘ │
    │       │         │ │       │         │ │       │         │
    │ ┌─────▼─────┐   │ │ ┌─────▼─────┐   │ │ ┌─────▼─────┐   │
    │ │ LH │SH│YTH │   │ │ │ Process   │   │ │ │   Info    │   │
    │ └───────────┘   │ │ │   Query   │   │ │ │ Provider  │   │
    │       │         │ │ └─────────────┘ │ │ └─────────────┘ │
    │ ┌─────▼─────┐   │ │       │         │ │                 │
    │ │  Display  │   │ │ ┌─────▼─────┐   │ │                 │
    │ │ & Process │   │ │ │ Generate  │   │ │                 │
    │ └───────────┘   │ │ │ Response  │   │ │                 │
    │                 │ │ └───────────┘   │ │                 │
    └─────────────────┘ └─────────────────┘ └─────────────────┘
              │                 │                 │
              ▼                 ▼                 ▼
    ┌─────────────────────────────────────────────────────────────────────┐
    │                      🗃️ NoticeState                                 │
    │                    (전역 상태 관리 객체)                              │
    │  • messages          • search_params      • selected_notice        │
    │  • LH_notice         • SH_notice          • YOUTH_notice           │
    │  • notices_displayed • waiting_for_selection                       │
    └─────────────────────────────────────────────────────────────────────┘
    """
    
    print(graph_structure)
    
    # 데이터 흐름 설명
    print("\n🔄 데이터 흐름:")
    print("-" * 40)
    flows = [
        "1️⃣ 사용자 입력 → Main Supervisor 분석",
        "2️⃣ 키워드 기반 라우팅 결정",
        "3️⃣ 적절한 Subgraph 실행",
        "4️⃣ NoticeState 상태 업데이트",
        "5️⃣ 결과 반환 및 다음 단계 준비"
    ]
    
    for flow in flows:
        print(f"  {flow}")
    
    print("\n🔧 주요 컴포넌트:")
    print("-" * 40)
    components = {
        "Main Supervisor": "중앙 제어, 입력 분석, 라우팅",
        "Subgraph 1": "LH/SH/YOUTH 스크래핑, 공고문 선택",
        "Subgraph 2": "공고문 검색, 질의 처리, 응답 생성",
        "Subgraph 3": "일반 대화, 기본 정보 제공",
        "NoticeState": "전역 상태 관리, 데이터 공유"
    }
    
    for component, description in components.items():
        print(f"  🔸 {component}: {description}")

def print_detailed_structure():
    """상세한 시스템 구조 출력"""
    print("\n" + "="*70)
    print("🏗️  상세 시스템 구조")
    print("="*70)
    
    print("\n📍 Main Graph:")
    print("  🎯 main_supervisor")
    print("    ├── 사용자 입력 분석")
    print("    ├── 키워드 기반 라우팅 결정")
    print("    └── 적절한 서브그래프로 전달")
    
    print("\n📍 Subgraph 1 (공고문 스크래핑):")
    print("  🎯 supervisor (에이전트 선택)")
    print("  🏢 LH_node (한국토지주택공사)")
    print("  🏙️ SH_node (서울주택도시공사)")
    print("  👥 YOUTH_node (청년안심주택)")
    print("  📊 display_notices (목록 표시)")
    print("  ✅ process_selection (선택 처리)")
    
    print("\n📍 Subgraph 2 (RAG 파이프라인):")
    print("  🔍 retrieve (내용 검색)")
    print("  ❓ process_query (질의 처리)")
    print("  💬 generate (응답 생성)")
    
    print("\n📍 Subgraph 3 (대화 처리):")
    print("  💭 chat (일반 대화)")
    print("  ℹ️ info_provider (기본 정보)")
    
    print("\n📊 상태 관리:")
    print("  🗃️ NoticeState")
    print("    ├── messages: 대화 메시지")
    print("    ├── task: 현재 작업")
    print("    ├── search_params: 검색 조건")
    print("    ├── LH_notice, SH_notice, YOUTH_notice: 스크래핑 결과")
    print("    ├── selected_notice: 선택된 공고문")
    print("    └── waiting_for_selection: 선택 대기 상태")

if __name__ == "__main__":
    # 예시 실행
    run_example()
    
    # LangGraph 내장 시각화 방법들
    visualize_graph()
    
    # 파일로 저장
    save_graph_visualization()
    
    # 커스텀 시각화
    create_custom_visualization()
    
    # 상세 구조 출력
    print_detailed_structure()

🏠 임대 주택 청약 도우미 시스템 시작

📋 시나리오 1: 공고문 검색
----------------------------------------
🎯 Main Supervisor: 사용자 입력 분석 중...
✅ main_supervisor 완료
📋 Notice Scrap Supervisor: 스크래핑 에이전트 선택 중...
👥 YOUTH Node: 청년안심주택 공고문 스크래핑 중...
✅ YOUTH에서 2개 공고문 수집 완료
📊 Display Notices: 공고문 목록 표시 중...
총 2개의 공고문을 찾았습니다.
1. [안심주택] 청년안심주택 2024년 1차 입주자 모집 (서울특별시)
   📅 2024-01-25 ~ 2024-02-25
2. [안심주택] 청년안심주택 강남구 신규 공급 (서울특별시 강남구)
   📅 2024-01-27 ~ 2024-02-27
✅ Process Selection: 사용자 선택 처리 중...
선택된 공고: 청년안심주택 2024년 1차 입주자 모집
✅ notice_scrap 완료

시스템 실행 완료!

🎨 LangGraph 시각화 방법들:

📊 방법 1: Mermaid 다이어그램 (텍스트)
----------------------------------------
%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	main_supervisor(main_supervisor)
	notice_scrap(notice_scrap)
	knowledge_rag(knowledge_rag)
	notice_rag(notice_rag)
	__end__([<p>__end__</p>]):::last
	__start__ --> main_supervisor;
	knowledge_rag --> __end__;
	notice_rag --> __end__;
	notice_scrap --> __end__;
	main_supervisor -.-> no